# 🧹 Learn Polars Data Cleaning - Interactive Notebook

Welcome to the **Agency Data Onboarding Kit** learning experience!

In this notebook, you'll learn how to clean messy client data using **Polars** - a blazing-fast data manipulation library. By the end, you'll understand exactly how to:

- 📊 Load and inspect messy CSV files
- 🧼 Normalize column names and values
- 🔍 Extract domains from website URLs
- 🌍 Standardize country names
- 📞 Clean phone numbers
- 🗑️ Remove duplicates intelligently
- ✅ Filter out invalid data
- 💾 Export clean data

---

## 🎯 How to Use This Notebook

1. **Run each cell in order** (Shift+Enter or click the Play button)
2. **Read the explanations** before each code block
3. **Experiment!** Change values and see what happens
4. **Use your own data** in the final section

**Time commitment:** 15-20 minutes

---

## 📚 What is Polars?

Polars is a modern data manipulation library that:
- Runs 5-10x faster than Pandas
- Uses less memory
- Has syntax that reads like plain English
- Handles messy data gracefully

Think of it as "Excel formulas that actually make sense."

---

**Ready? Let's dive in!** 🚀

## Step 1: Install Polars

First, we need to install the Polars library. This only takes a few seconds.

In [ ]:
!pip install polars -q

print("✅ Polars installed successfully!")

## Step 2: Import Libraries

Let's import Polars and set up our environment.

In [ ]:
import polars as pl
import re
from datetime import datetime

# Configure Polars to show more rows in output
pl.Config.set_tbl_rows(20)

print("✅ Libraries imported!")
print(f"📦 Polars version: {pl.__version__}")

## Step 3: Load Sample Data

We'll download a sample messy CSV file from the repository. This is what a typical client sends: chaotic, inconsistent, and full of duplicates.

**What's in this file:**
- 41 contact records
- Duplicate emails with different data completeness
- Mixed phone formats
- Country variations (UK, GB, United Kingdom, etc.)
- Invalid emails
- Missing data

In [ ]:
# Download sample messy contacts CSV
url = "https://raw.githubusercontent.com/billion-community/agency-data-onboarding-kit/main/samples/contacts_messy.csv"

# Load the CSV
df_raw = pl.read_csv(url)

print(f"✅ Loaded {len(df_raw)} rows")
print(f"\n📋 Columns: {df_raw.columns}")
print("\n🔍 First 5 rows:")
df_raw.head(5)

### 👀 Inspect the Chaos

Look at the data above. Notice:
- **Trailing spaces** in column names (`"Title "`, `"Country "`)
- **Mixed case emails** (SARAH.JOHNSON@... vs sarah.johnson@...)
- **Different phone formats** ((555) 123-4567 vs +1-555-234-5678)
- **Country variations** (USA, United States, US)

Let's fix all of this automatically! 💪

## Step 4: Normalize Column Names

First issue: column names have trailing spaces and inconsistent formatting.

**Goal:** Convert all columns to lowercase with underscores (snake_case)

**Example:** `"Full Name"` → `"full_name"`, `"Title "` → `"title"`

In [ ]:
# Before: see the messy column names
print("❌ BEFORE:")
print(df_raw.columns)

# Normalize column names
df = df_raw.rename({
    col: col.strip().lower().replace(" ", "_") 
    for col in df_raw.columns
})

# After: clean column names
print("\n✅ AFTER:")
print(df.columns)

print("\n🎉 All columns are now clean and consistent!")

### 💡 What Just Happened?

```python
col.strip()           # Remove leading/trailing spaces
   .lower()           # Convert to lowercase
   .replace(" ", "_") # Replace spaces with underscores
```

This is a **dictionary comprehension** - it creates a mapping of old names → new names for all columns at once.

## Step 5: Clean Email Addresses

Emails should always be:
- Lowercase (sarah.johnson@acme.com, not SARAH.JOHNSON@ACME.COM)
- Trimmed (no leading/trailing spaces)

Let's fix this with Polars!

In [ ]:
# See the problem
print("❌ BEFORE - Mixed case emails:")
print(df.select(["full_name", "email"]).head(3))

# Clean emails
df = df.with_columns([
    pl.col("email").str.to_lowercase().str.strip().alias("email")
])

print("\n✅ AFTER - All lowercase:")
print(df.select(["full_name", "email"]).head(3))

### 💡 Understanding Polars Syntax

```python
df.with_columns([          # Add or modify columns
    pl.col("email")         # Select the email column
      .str.to_lowercase()   # Make it lowercase
      .str.strip()          # Remove spaces
      .alias("email")       # Keep the same column name
])
```

This reads like English: "Take the email column, make it lowercase, strip spaces, and save it back as email."

## Step 6: Extract Domains from Emails

We need to know which company each contact belongs to. The domain in their email is a great indicator.

**Example:** `sarah.johnson@acme-corp.com` → `acme-corp.com`

In [ ]:
# Extract domain from email
df = df.with_columns([
    pl.col("email")
      .str.split("@")
      .list.get(1)  # Get the part after @
      .alias("email_domain")
])

print("✅ Domains extracted:")
print(df.select(["email", "email_domain"]).head(5))

## Step 7: Normalize Country Names

Look at the country column - it's a mess:
- USA, United States, US
- UK, United Kingdom, GB, U.K., uk

Let's standardize these!

In [ ]:
# First, see the problem
print("❌ BEFORE - Country variations:")
print(df.group_by("country").count().sort("count", descending=True))

# Define mapping
country_map = {
    "usa": "United States",
    "us": "United States",
    "united states": "United States",
    "uk": "United Kingdom",
    "gb": "United Kingdom",
    "united kingdom": "United Kingdom",
    "u.k.": "United Kingdom",
}

# Normalize countries
df = df.with_columns([
    pl.col("country")
      .str.to_lowercase()
      .str.strip()
      .replace(country_map)
      .alias("country")
])

print("\n✅ AFTER - Standardized:")
print(df.group_by("country").count().sort("count", descending=True))

## Step 8: Clean Phone Numbers

Phone numbers come in many formats. Let's keep only digits and the leading `+` sign.

In [ ]:
# See the chaos
print("❌ BEFORE - Phone number chaos:")
print(df.select(["full_name", "phone"]).head(10))

# Clean phone numbers
def clean_phone(phone):
    if phone is None or phone == "":
        return None
    # Keep only digits and leading +
    if phone.startswith("+"):
        return "+" + re.sub(r"[^0-9]", "", phone)
    else:
        return re.sub(r"[^0-9]", "", phone)

df = df.with_columns([
    pl.col("phone").map_elements(clean_phone, return_dtype=pl.Utf8).alias("phone")
])

print("\n✅ AFTER - Clean phone numbers:")
print(df.select(["full_name", "phone"]).head(10))

## Step 9: Clean LinkedIn URLs

Let's standardize LinkedIn URLs to a consistent format.

In [ ]:
# Standardize LinkedIn URLs
def clean_linkedin(url):
    if url is None or url == "":
        return None
    url = url.lower()
    url = url.replace("https://", "").replace("http://", "")
    url = url.replace("www.", "")
    if not url.startswith("linkedin.com"):
        return None
    return f"https://{url}"

df = df.with_columns([
    pl.col("linkedin").map_elements(clean_linkedin, return_dtype=pl.Utf8).alias("linkedin")
])

print("✅ LinkedIn URLs standardized")

## Step 10: Filter Out Invalid Emails

Some rows have invalid emails. Let's remove them.

In [ ]:
print(f"❌ BEFORE filtering: {len(df)} rows")

# Show invalid emails
invalid = df.filter(
    ~pl.col("email").str.contains("@") | 
    pl.col("email").is_null()
)
print(f"\n⚠️ Found {len(invalid)} invalid emails")

# Filter them out
df = df.filter(
    pl.col("email").str.contains("@") & 
    pl.col("email").is_not_null()
)

# Also filter out test/generic emails
df = df.filter(
    ~pl.col("email").str.contains("test@") &
    ~pl.col("email").str.starts_with("info@")
)

print(f"\n✅ AFTER filtering: {len(df)} rows")
print(f"🗑️ Removed {len(df_raw) - len(df)} invalid rows")

## Step 11: Calculate Data Completeness Score

Let's score each row based on how much information it has.

In [ ]:
# Calculate completeness score
df = df.with_columns([
    (
        pl.col("full_name").is_not_null().cast(pl.Int32) +
        pl.col("email").is_not_null().cast(pl.Int32) +
        pl.col("title").is_not_null().cast(pl.Int32) +
        pl.col("phone").is_not_null().cast(pl.Int32) +
        pl.col("linkedin").is_not_null().cast(pl.Int32)
    ).alias("completeness_score")
])

print("📊 Data Completeness Distribution:")
print(df.group_by("completeness_score").count().sort("completeness_score", descending=True))

## Step 12: Identify Duplicates

Let's find duplicate emails in our dataset.

In [ ]:
# Find duplicated emails
duplicates = df.filter(pl.col("email").is_duplicated())

print(f"⚠️ Found {len(duplicates)} duplicate email records")
if len(duplicates) > 0:
    print("\n🔍 Duplicates (sorted by email):")
    print(duplicates.sort("email").select([
        "email", "full_name", "title", "phone", "completeness_score"
    ]))

## Step 13: Deduplicate (Keep Best Record)

Now the magic! We'll keep only the most complete record for each email.

In [ ]:
print(f"❌ BEFORE deduplication: {len(df)} rows")

# Deduplicate by email, keeping the most complete record
df_clean = (
    df.sort("completeness_score", descending=True)
      .unique(subset=["email"], keep="first")
      .drop("completeness_score")
)

print(f"✅ AFTER deduplication: {len(df_clean)} rows")
print(f"🗑️ Removed {len(df) - len(df_clean)} duplicate records")

# Verify no more duplicates
remaining_dupes = df_clean.filter(pl.col("email").is_duplicated())
print(f"\n✨ Remaining duplicates: {len(remaining_dupes)} (should be 0!)")

## Step 14: Final Data Quality Check

Let's see what we've accomplished!

In [ ]:
print("📊 FINAL DATA QUALITY REPORT")
print("=" * 50)

print(f"\n✅ Total clean records: {len(df_clean)}")
print(f"📧 All emails valid: {df_clean.filter(pl.col('email').str.contains('@')).height == len(df_clean)}")
print(f"🔍 No duplicate emails: {df_clean.filter(pl.col('email').is_duplicated()).height == 0}")

print("\n📈 Field Completeness:")
for col in ["full_name", "email", "title", "phone", "linkedin", "country"]:
    non_null = df_clean.filter(pl.col(col).is_not_null()).height
    pct = (non_null / len(df_clean)) * 100
    print(f"  {col:15s}: {non_null:3d}/{len(df_clean):3d} ({pct:5.1f}%)")

print("\n🌍 Records by Country:")
print(df_clean.group_by("country").count().sort("count", descending=True))

## Step 15: Preview Clean Data

Let's look at the final, beautiful, clean data!

In [ ]:
print("✨ YOUR CLEAN DATA (first 10 rows):\n")
df_clean.head(10)

## Step 16: Export Clean Data

Time to save your clean data!

In [ ]:
# Export to CSV
output_filename = f"contacts_clean_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df_clean.write_csv(output_filename)

print(f"✅ Clean data exported to: {output_filename}")
print(f"📦 File size: {len(df_clean)} rows x {len(df_clean.columns)} columns")
print("\n💾 Download the file from the files panel on the left →")

print("\n📊 TRANSFORMATION SUMMARY:")
print("=" * 50)
print(f"  Original rows:        {len(df_raw)}")
print(f"  Invalid filtered:     {len(df_raw) - len(df)}")
print(f"  Duplicates removed:   {len(df) - len(df_clean)}")
print(f"  Final clean rows:     {len(df_clean)}")
print(f"  Data retained:        {(len(df_clean)/len(df_raw)*100):.1f}%")

## 🎓 What You Just Learned

Congratulations! You just cleaned messy client data like a pro.

### Core Polars Skills
✅ Loading, transforming, filtering, sorting, deduplicating data  
✅ String operations and data type handling  
✅ Aggregations and quality checks  

### Data Cleaning Techniques
✅ Normalization, validation, intelligent deduplication  

---

## 🚀 Next Steps

1. ⭐ **Star the GitHub repo**: [agency-data-onboarding-kit](https://github.com/billion-community/agency-data-onboarding-kit)
2. 💬 **Join the WhatsApp community** - Share results, get help
3. 🔨 **Build the full pipeline** - Add n8n + Supabase automation
4. 📧 **Subscribe to newsletter** - Advanced techniques

---

*Built with ❤️ by the Billion community*